In [1]:
CODE_REVIEWER = """
# IDENTITY

You are an AI named Codia. You have extensive knowledge and skill in programming languages, especially Python. You are aware of the best practices used in programming, have an extensive extensive experience in lagorithms, data structures and overall computer science.

You are a concise expert in evaluating and refining the code generated by an AI assistant based on a Large Language Model.

# GOALS

Your task is to evaluate and provide feedback for a conversation between a human user and an AI Assistant that is based on the latest large language model architecture.
Focus of your evaluation is code in the replies generated by the AI Assistant only. The conversation environment is a Jupyter notebook, thus things that are run in other cells, are available in the next cells.

# RULES

Attributes to consider:
- Code Correctness
- Code Efficiency
- Best Practices
- Code Readability
- Code style Consistency

**1. Identification of Code for Review**
- Target for analysis: Code generated by the LLM Assistant in a reply to the User within a Jupyter notebook exchange.
- Exclude analysis of human user input for focused improvement on LLM-generated content.
- Exclude LLM Assistant text content, only review code snippets and code cells. Text is for context and reasoning/explanation only.
- Additional code cells and interactions may precede or follow the one under review, providing context for the analysis.
- Exclude concerns about code explanation in the text parts if they are not comments inside the code, as it will be covered by other reviewers.

**2. Evaluation Criteria Definitions**
- Correctness: The code must be devoid of bugs and errors.
- Efficiency: The code must be optimized for maximum performance.
- Best Practices: The code must adhere to established programming conventions, techniques, and guidelines.
- Readability: The code must be easily comprehensible, with suitable naming conventions and comments where complexity demands.
- Consistency: The code must be consistent with the Assistant's programming identity and the context of the user interaction.

**3. Review Guidelines**
- Avoid general praise observations: Be specific and objective in your feedback.
- Avoid nitpicky/subjective criticism: Focus on substantial issues that affect the code quality.

# OUTPUT_FORMAT

Your output must be a JSON object with the following fields:

# critical_severity_issues - list of issues that make the conversation not useful for the human user.
# medium_severity_issues - list of issues that still have a strong influence on the conversation flow and usefulness.
Each issue should point out a specific problem and how to fix it in exec summary fashion. Start each issue with the cell position it was found in and a short piece of text to find specific place of the issue.
Each issue consists of an `issue_dict` - should contain the following fields: `cell_position`, `what` is the issue, `why` it's an issue, `where` in the cell, how to `fix` it.

# scoring_explanation - using rubrics, concisly point out the logic behind scoring this concersation.

# score - a number between 1 and 5 that ranges from the quality of the code, where 1 is the worst and 5 is the best, based on the criteria outlined in the grading rubric.

### 5 - Excellent
- Well Formatted
- Correct
- Optimal
- Highly readable

### 4 - Good
- Correct but can be slightly optimized in terms of approach / speed / readability

### 3 - Acceptable
- The code is correct but can be significantly improved.
- The code is not readable.

### 2 - Needs Improvement
- The code is incorrect / out of scope / has syntax errors.
- Looks like it’s copied from ChatGPT.

### 1 - Poor
- Incomplete or Code required by context of the interaction but missing.

```json
{
    "critical_severity_issues": list[issue_dict],
    "medium_severity_issues": list[issue_dict],
    "scoring_explanation": str,
    "score": number
}
```

# REFOCUS:
- You are a code reviewer, not a language and contextual information content reviewer. Do not mention issues not related to your purpose.
- If the code was **unnecessary** it can be absent and thus must receive highest score - nothing to review = highest score.
"""

In [3]:
import utils

from src.llm_reviewer.llm_api import make_llm_request, LLMAPIFactory
import os
import sys
from src.llm_reviewer.notebook_reviewer import IssueLevel
from src.llm_reviewer.llm_api import global_usage_manager
global_usage_manager.reset_usage()

/home/saya/chario/upstream_character_tasks/.venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:

def make_review_request(nb):
    llm_client = LLMAPIFactory().get()
    cells = [{'type': cell['cell_type'], 'content': cell['source']} for cell in nb['cells']]
    cells_with_numbers = [{'cell_position': i + 1, **c,} for i, c in enumerate(cells)]
    r = make_llm_request(
        llm_client,
        [{'role': 'system', 'content': CODE_REVIEWER}, {'role': 'user', 'content': f'# CONVERSATION_START\n\n\n{cells_with_numbers}\n\n\n# CONVERSATION_END'}],
        'gpt-4-1106-preview',
        temperature=0.0,
        max_tokens=4000,
        response_format={"type": "json_object"},
        retries=3,
        seed=42
    )
    return r

In [ ]:
from IPython.display import display, Markdown

def format_review_sections(data):
    sections = ["critical_severity_issues", "medium_severity_issues", "scoring_explanation", "score"]
    markdown_output = ""
    for section in sections:
        if section in data:
            if isinstance(data[section], list):
                markdown_output += f"# {section.replace('_', ' ').title()}\n"
                for issue in data[section]:
                    issue_details = f"**Cell {issue['cell_position']}**: {issue['what']}\n"
                    issue_details += f"- **Why**: {issue['why']}\n"
                    issue_details += f"- **Where**: {issue['where']}\n"
                    issue_details += f"- **Fix**: {issue['fix']}\n\n"
                    markdown_output += issue_details
            else:
                if section == "score":
                    markdown_output += f"# Score: {data[section]}\n\n"
                else:
                    markdown_output += f"# {section.replace('_', ' ').title()}\n{data[section]}\n\n"
        else:
            raise Exception(f'{section=} not found')
    return markdown_output

def print_as_markdown(data):
    markdown_output = format_review_sections(data)
    display(Markdown(markdown_output))


In [ ]:
r = make_review_request(notebook)
print_as_markdown(r)

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timezone
import pandas as pd

# Use your own service account file
service_account_file = '/home/ubuntu/Turing/character_tasks/creds/google__sa.json'

# Define the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Create a service account credential
creds = service_account.Credentials.from_service_account_file(service_account_file, scopes=SCOPES)

# Build the service
drive_service = build('drive', 'v3', credentials=creds)

def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    timestamp = pd.to_datetime(timestamp).date()

    # Initialize the latest revision before the timestamp as None
    latest_revision_before_timestamp = None

    for revision in revisions['revisions']:
        # Convert the modifiedTime of the revision to a datetime object
        modified_time = pd.to_datetime(revision['modifiedTime']).date()
        # If the modifiedTime is before the timestamp
        if modified_time <= timestamp:
            # If this is the first revision or this revision is later than the latest found so far
            if latest_revision_before_timestamp is None or modified_time > pd.to_datetime(latest_revision_before_timestamp['modifiedTime']).date():
                # Update the latest revision before the timestamp
                latest_revision_before_timestamp = revision

    return latest_revision_before_timestamp

# # Example usage:
# file_id = '15AXjZoIhlO9GZl9BTKxSh9JHJkIl_Qh2'
# timestamp = '12/20/2023'
# timestamp = pd.to_datetime(timestamp).date()
# revision = get_revision_before_timestamp(file_id, timestamp)
# print("revision :" , revision)

# def print_all_revision_ids(file_id):
#     try:
#         # Get the revisions of the file
#         revisions = drive_service.revisions().list(fileId=file_id).execute()
#     except Exception as e:
#         print(f"File not found: {file_id}")
#         return None

#     # Print all revision ids
#     for revision in revisions['revisions']:
#         print(revision)

# # Example usage:
# file_id = '15AXjZoIhlO9GZl9BTKxSh9JHJkIl_Qh2'
# print_all_revision_ids(file_id)


def get_file_id_from_task_link(task_link):
    try:
        return task_link.split("/")[-1]
    except Exception as e:
        print('ERROR' + '='*60)
        print(task_link)
        return None

# Add a new column to the DataFrame for the file IDs
selected_rows_df['file_id'] = selected_rows_df['Task Link [Google Colab]'].apply(get_file_id_from_task_link)

# Apply the function to each row in the DataFrame
selected_rows_df['revision'] = selected_rows_df.apply(lambda row: get_revision_before_timestamp(row['file_id'], row['Timestamp']) if row['file_id'] is not None else None, axis=1)

In [ ]:
def get_revision_before_timestamp(file_id, timestamp):
    try:
        # Get the revisions of the file
        revisions = drive_service.revisions().list(fileId=file_id).execute()
    except Exception as e:
        print(f"File not found: {file_id}")
        return None

    # Convert the timestamp to a datetime object
    timestamp = pd.to_datetime(timestamp).date()

In [ ]:
import 